<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEkphrasisnegpos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df_train.sample(frac=1)
df_test = df_train.sample(frac=1)

Mounted at /content/drive
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...

In [ ]:
df_train.head(1000)

,Tweet,Emotion
8259,tri to spend time away from the realitybrok my...,2
942,how do you twitter,0
11148,my poor laptop got a quotrogu malwarequot visi...,2
5814,ouch my back man I am sick,2
6527,cannot believ thi shit ive spend minut for for...,2
...,...,...
10846,oop correct it nowheremonastri cannot seem to...,0
9749,i miss my boo on anoth note I am soreadi for t...,2
10900,icebergmeadow no but thi is our poor week,2
10805,supergirln sorri all the code i have are post ...,2


In [ ]:
df_test.Emotion.unique()

array(['1', '2', '0', '3', 'emotion'], dtype=object)

In [ ]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [ ]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [ ]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [ ]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(1303926, 1724630)

In [ ]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [ ]:
positive = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/positive-words.csv')
positive = list(positive['0'][1:])
negative = pd.read_csv('/content/drive/My Drive/InfluenceAnalysis/CodeMix/negative-words.csv')
negative = list(negative['0'][2:])
pn_dict = {}
for i in range(len(positive)):
  pn_dict[positive[i]] = 1

for i in range(len(negative)):
  pn_dict[negative[i]] = -1

In [ ]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
X_train = [nltk.pos_tag(x) for x in X_train]
X_test = [nltk.pos_tag(x) for x in X_test]
X_train = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_train]
X_test = [[(i[0],i[1],pn_dict.get(i[0],1)) for i in x] for x in X_test]
result_table = [0,0,0,0,0,0,0,0,0]

In [ ]:
print(X_train[0][0])

('tri', 'NN', 1)


In [ ]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.get_vector(i[0]),i[1],i[2]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            tweet[i]=((tweet[i][0]*tweet[i][2]*mul_factor),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1],i[2]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ':
            tweet[i]=((tweet[i][0]*tweet[i][2]*mul_factor),tweet[i][1],tweet[i][2])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs = 20

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
24/24 [==============================] - 5s 217ms/step - loss: 1.2397 - accuracy: 0.5685
Epoch 2/20
24/24 [==============================] - 5s 223ms/step - loss: 1.0086 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 5s 225ms/step - loss: 0.9834 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 5s 219ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 5s 225ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 5s 221ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 5s 221ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 5s 223ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 10/20
24/24 [=====

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

Epoch 1/20
24/24 [==============================] - 5s 221ms/step - loss: 1.2277 - accuracy: 0.5772
Epoch 2/20
24/24 [==============================] - 5s 222ms/step - loss: 1.0106 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 5s 227ms/step - loss: 0.9841 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 230ms/step - loss: 0.9838 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9837 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9835 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 5s 223ms/step - loss: 0.9834 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9833 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9831 - accuracy: 0.5863

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("SG_RNN")

Epoch 1/20
24/24 [==============================] - 6s 231ms/step - loss: 1.0888 - accuracy: 0.5372
Epoch 2/20
24/24 [==============================] - 6s 234ms/step - loss: 0.9860 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9847 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9847 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 6s 232ms/step - loss: 0.9848 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 6s 234ms/step - loss: 0.9849 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9849 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 6s 231ms/step - loss: 0.9850 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9850 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 6s 235ms/step - loss: 0.9850 - accuracy: 0.5863

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

Epoch 1/20
24/24 [==============================] - 5s 215ms/step - loss: 1.2405 - accuracy: 0.5732
Epoch 2/20
24/24 [==============================] - 5s 218ms/step - loss: 1.0091 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 5s 220ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 5s 224ms/step - loss: 0.9841 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 5s 221ms/step - loss: 0.9840 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 5s 220ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 5s 220ms/step - loss: 0.9839 - accuracy: 0.5863

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

Epoch 1/20
24/24 [==============================] - 5s 221ms/step - loss: 1.2326 - accuracy: 0.5713
Epoch 2/20
24/24 [==============================] - 5s 224ms/step - loss: 1.0109 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 5s 225ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9839 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 5s 223ms/step - loss: 0.9837 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9836 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9834 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 5s 222ms/step - loss: 0.9833 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 5s 217ms/step - loss: 0.9831 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 5s 224ms/step - loss: 0.9829 - accuracy: 0.5863

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

Epoch 1/20
24/24 [==============================] - 6s 231ms/step - loss: 1.0689 - accuracy: 0.5699
Epoch 2/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9849 - accuracy: 0.5863
Epoch 3/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9844 - accuracy: 0.5863
Epoch 4/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9843 - accuracy: 0.5863
Epoch 5/20
24/24 [==============================] - 6s 249ms/step - loss: 0.9842 - accuracy: 0.5863
Epoch 6/20
24/24 [==============================] - 6s 246ms/step - loss: 0.9842 - accuracy: 0.5863
Epoch 7/20
24/24 [==============================] - 6s 235ms/step - loss: 0.9842 - accuracy: 0.5863
Epoch 8/20
24/24 [==============================] - 5s 229ms/step - loss: 0.9842 - accuracy: 0.5863
Epoch 9/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9842 - accuracy: 0.5863
Epoch 10/20
24/24 [==============================] - 6s 232ms/step - loss: 0.9842 - accuracy: 0.5863

In [ ]:
epochs=40
Embedding_size=100

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


Epoch 1/40
24/24 [==============================] - 1s 35ms/step - loss: 1.2074 - accuracy: 0.5595
Epoch 2/40
24/24 [==============================] - 1s 36ms/step - loss: 1.0011 - accuracy: 0.5863
Epoch 3/40
24/24 [==============================] - 1s 37ms/step - loss: 0.9772 - accuracy: 0.5864
Epoch 4/40
24/24 [==============================] - 1s 37ms/step - loss: 0.9699 - accuracy: 0.5864
Epoch 5/40
24/24 [==============================] - 1s 38ms/step - loss: 0.9667 - accuracy: 0.5864
Epoch 6/40
24/24 [==============================] - 1s 36ms/step - loss: 0.9621 - accuracy: 0.5873
Epoch 7/40
24/24 [==============================] - 1s 36ms/step - loss: 0.9569 - accuracy: 0.5915
Epoch 8/40
24/24 [==============================] - 1s 36ms/step - loss: 0.9502 - accuracy: 0.5943
Epoch 9/40
24/24 [==============================] - 1s 36ms/step - loss: 0.9440 - accuracy: 0.5957
Epoch 10/40
24/24 [==============================] - 1s 36ms/step - loss: 0.9382 - accuracy: 0.5984
Epoch 11/

In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

Epoch 1/40
24/24 [==============================] - 1s 37ms/step - loss: 1.0763 - accuracy: 0.5594
Epoch 2/40
24/24 [==============================] - 1s 38ms/step - loss: 0.9691 - accuracy: 0.5863
Epoch 3/40
24/24 [==============================] - 1s 39ms/step - loss: 0.9499 - accuracy: 0.5878
Epoch 4/40
24/24 [==============================] - 1s 39ms/step - loss: 0.9318 - accuracy: 0.5946
Epoch 5/40
24/24 [==============================] - 1s 39ms/step - loss: 0.9165 - accuracy: 0.6003
Epoch 6/40
24/24 [==============================] - 1s 38ms/step - loss: 0.9021 - accuracy: 0.6063
Epoch 7/40
24/24 [==============================] - 1s 39ms/step - loss: 0.8879 - accuracy: 0.6131
Epoch 8/40
24/24 [==============================] - 1s 38ms/step - loss: 0.8770 - accuracy: 0.6174
Epoch 9/40
24/24 [==============================] - 1s 39ms/step - loss: 0.8659 - accuracy: 0.6250
Epoch 10/40
24/24 [==============================] - 1s 38ms/step - loss: 0.8549 - accuracy: 0.6323
Epoch 11/

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/40
24/24 [==============================] - 1s 52ms/step - loss: 1.1004 - accuracy: 0.5547
Epoch 2/40
24/24 [==============================] - 1s 50ms/step - loss: 0.9853 - accuracy: 0.5864
Epoch 3/40
24/24 [==============================] - 1s 51ms/step - loss: 0.9839 - accuracy: 0.5864
Epoch 4/40
24/24 [==============================] - 1s 52ms/step - loss: 0.9818 - accuracy: 0.5866
Epoch 5/40
24/24 [==============================] - 1s 50ms/step - loss: 0.9813 - accuracy: 0.5864
Epoch 6/40
24/24 [==============================] - 1s 51ms/step - loss: 0.9782 - accuracy: 0.5863
Epoch 7/40
24/24 [==============================] - 1s 50ms/step - loss: 0.9725 - accuracy: 0.5871
Epoch 8/40
24/24 [==============================] - 1s 50ms/step - loss: 0.9682 - accuracy: 0.5895
Epoch 9/40
24/24 [==============================] - 1s 51ms/step - loss: 0.9660 - accuracy: 0.5909
Epoch 10/40
24/24 [==============================] - 1s 49ms/step - loss: 0.9614 - accuracy: 0.5911
Epoch 11/

In [ ]:
model_sg1 = tf.keras.Sequential()
model_sg1.add(tf.keras.Input(shape=(Max_input_size,100)))
model_sg1.add(tf.keras.layers.LSTM(64))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg1.add(tf.keras.layers.Dropout(0.2))
model_sg1.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg1.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg1.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg1.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")

Epoch 1/40
24/24 [==============================] - 5s 216ms/step - loss: 1.2185 - accuracy: 0.5727
Epoch 2/40
24/24 [==============================] - 5s 220ms/step - loss: 1.0274 - accuracy: 0.5862
Epoch 3/40
24/24 [==============================] - 5s 220ms/step - loss: 0.9973 - accuracy: 0.5859
Epoch 4/40
24/24 [==============================] - 5s 224ms/step - loss: 0.9955 - accuracy: 0.5862
Epoch 5/40
24/24 [==============================] - 5s 221ms/step - loss: 0.9957 - accuracy: 0.5864
Epoch 6/40
24/24 [==============================] - 5s 221ms/step - loss: 0.9936 - accuracy: 0.5863
Epoch 7/40
24/24 [==============================] - 5s 219ms/step - loss: 0.9892 - accuracy: 0.5863
Epoch 8/40
24/24 [==============================] - 6s 229ms/step - loss: 0.9915 - accuracy: 0.5863
Epoch 9/40
24/24 [==============================] - 5s 225ms/step - loss: 0.9890 - accuracy: 0.5863
Epoch 10/40
24/24 [==============================] - 5s 229ms/step - loss: 0.9873 - accuracy: 0.5863

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.5845000147819519, 0.5845000147819519, 0.5845000147819519]
cbow [0.5845000147819519, 0.5845000147819519, 0.5845000147819519]
glove [0.7239166498184204, 0.7700833082199097, 0.6159166693687439]


In [ ]:
print(result_table[9])

0.5845000147819519
